In [ ]:
import pandas as pd

In [ ]:
sales = pd.read_csv('m5-forecasting-accuracy/sales_train_evaluation.csv')
sales

In [ ]:
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
calendar

In [ ]:
# Melt sales data (convert from wide to long format)
sales_long = sales.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], var_name="d", value_name="y")

# Merge with calendar to get actual dates
sales_long = sales_long.merge(calendar[["d", "date"]], on="d", how="left")

# Create unique_id
sales_long["unique_id"] = sales_long["item_id"] + "_" + sales_long["store_id"]

# Rename columns to match StemGNN requirements
sales_long = sales_long.rename(columns={"date": "ds"})[["unique_id", "ds", "y"]]

# Sort data
sales_long = sales_long.sort_values(by=["unique_id", "ds"])

sales_long

In [ ]:
sales_long.to_csv("m5_stemgnn.csv", index=False)

In [ ]:
def has_30_consecutive_zeros(y_values):
    count = 0
    for val in y_values:
        if val == 0:
            count += 1
            if count >= 30:
                return True  # Found 30 consecutive zeros
        else:
            count = 0  # Reset counter
    return False

In [ ]:
df = sales_long.copy()
# Identify `unique_id`s to remove
ids_to_remove = df.groupby("unique_id")["y"].apply(has_30_consecutive_zeros)
ids_to_remove = ids_to_remove[ids_to_remove].index  # Get unique_ids to drop
ids_to_remove

In [ ]:
# Filter dataset
df_filtered = df[~df["unique_id"].isin(ids_to_remove)]

# Save cleaned dataset
df_filtered.to_csv("m5_stemgnn_filtered.csv", index=False)

print(f"Removed {len(ids_to_remove)} unique_id(s) with 30 consecutive zeros.")

In [ ]:
def convert_daily_to_weekly(input_csv_path, output_csv_path):
    """
    Converts a daily sales dataset to weekly by summing sales over each week.
    The first day of the week (Saturday) is used as the timestamp.
    
    Parameters:
    input_csv_path (str): Path to the input daily CSV file.
    output_csv_path (str): Path to save the output weekly CSV file.
    """
    # Load daily data
    daily_df = pd.read_csv(input_csv_path)
    daily_df['ds'] = pd.to_datetime(daily_df['ds'])
    
    # Load calendar to get Walmart week numbers
    calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
    calendar['date'] = pd.to_datetime(calendar['date'])
    
    # Merge with calendar to obtain 'wm_yr_wk' (Walmart week identifier)
    merged = daily_df.merge(
        calendar[['date', 'wm_yr_wk']],
        left_on='ds', 
        right_on='date', 
        how='left'
    )
    
    # Create mapping from Walmart week to the first day of the week (Saturday)
    week_start_map = calendar.groupby('wm_yr_wk')['date'].min().reset_index()
    week_start_map.columns = ['wm_yr_wk', 'week_start_date']
    
    # Aggregate sales by unique_id and Walmart week
    weekly_sales = merged.groupby(['unique_id', 'wm_yr_wk'])['y'].sum().reset_index()
    
    # Add week start date (Saturday) as the new timestamp
    weekly_sales = weekly_sales.merge(
        week_start_map, 
        on='wm_yr_wk', 
        how='left'
    )
    
    # Format final output
    weekly_sales = weekly_sales[['unique_id', 'week_start_date', 'y']]
    weekly_sales.rename(columns={'week_start_date': 'ds'}, inplace=True)
    weekly_sales.sort_values(['unique_id', 'ds'], inplace=True)
    
    # Save weekly dataset
    weekly_sales.to_csv(output_csv_path, index=False)
    

In [ ]:
# Convert all datasets to weekly format
datasets = [
    ('m5_stemgnn.csv', 'm5_stemgnn_weekly.csv'),
    ('m5_stemgnn_filtered.csv', 'm5_stemgnn_filtered_weekly.csv'),
]

for input_file, output_file in datasets:
    convert_daily_to_weekly(input_file, output_file)